# NICK'S ORIGINAL CODE

# OBJECTIVE: (TEAM MEETING NOTES)

1. Complete EDA analysis on the base NER model. 
2. After we can begin experimenting with LLM models. 
3. Once we determine best model, we can move forward with explainability features that work best for our business use case. 

**Business use case: 
- individuals trying to explore the job market without industries paying for a first step forward. 
- A job searching tool that is completely unbiased from other companies, just looking forward with the individual’s best intention to aid in their job hunt. 


**FOR LATER:

Idea: 
- use API from Bureau of Labor Statistics to use as baseline of occupations and industries (free open source data to extract from) 
- Job recommendations idea: use cluster analysis to give consumer “jobs you may enjoy if you want to move industries(industries as series, job title as title and if we want to reference NAICS directly, we can reference the 6 digit NAICS code).


In [149]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
#from docx import Document
import os
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pdfplumber
import nltk
nltk.download('punkt')
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px
import os
import json
#from docx import Document
import pdfplumber
from bs4 import BeautifulSoup
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud
spacy.cli.download("en_core_web_sm")
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
from plotly.offline import plot
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dezri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [150]:
#!pip install transformers torch

In [151]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline

In [152]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

***

## Loading the Model and Tokenizer

In [153]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [154]:
resume_text = """

Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
Nicholas.E.Reese15@gmail.com 
Education 
•Georgetown University - August 2023 – December 2024
Master of Science Business Analytics – MSBA candidate
Peer-Elected Class Representative 
•Dickinson College - August 2014 – September 2018
Economics & Political Science Double Major
Varsity Tennis Captain 
•University of Bologna - August 2016

Skills
- R studio
- Python
- Power BI
- SQL
- Neural Networks
- Machine Learning
- Macro Modeling
- AWS Cloud Services
- Pandas
- A/B Testing
- Scikit-Learn
- Econometrics

Professional Experience
FINRA, Washington, D.C.	June 2020 – Present 
Senior Analyst, Market Regulation	September 2022 – Present 
•Earned top 10% of performance of analysts for past two years.
•Led team in research implementing PostgreSQL and NoSQL queries for large data pulls.
•Spearheaded new analytical approaches to financial workflow with tools such as R & Python for model development.
•Developed working predictive modeling schemas for senior staff using statistical analysis.
•Leveraged skills in platforms like Python, R, Power BI, Tableau and SQL accompanied with strong statistical background in financial markets.
•Built the Security-Based Swap training manual deck & produced the recorded info session for all of FINRA. 
•Selected for the FINRA’s first Georgetown Advanced Analytics Program as one of the most junior staff awarded opportunity.
•Incorporated statistical analytics to assist in creating a NPL tool to analyze financial documents language to minimize time on manual analysis.
•Produced unsupervised and supervised models to perform analysis for Security-Based Swaps trade patterns.
•Improved FINRA platforms with Data Scientists for higher accuracy and efficiency. 
•Managed process of re-engineering supervisory reviews through advanced analytic tools for senior staff.
•Implements advanced analytics to assist senior staff with maximizing efficiencies in daily workloads and trade pattern creations.
•Presented visualization case work using Power BI & Tableau to senior leadership.
•Persuaded senior staff to allow for statistical analytics tools like R.
•Mentored over 20 junior & senior staff members on improving processes with analytical tools for financial reviews.
•Managed junior staff with day-to-day workload while teaching the staff how to use advanced analytics.
Analyst, Market Regulation	June 2021-September 2022
·Created macroeconomic models for newly implemented FINRA Rule 2232 and MSRB Rule G-15.
·Mentored junior staff with creating macro models, synthesizing responses from FINRA member firms.
·Won the Regulator Scholarship for continued financial learning based on individual performance.
·Received six internal awards for expertise in Municipal and Corporate Bond analysis as an analyst.
Associate Analyst, Market Regulation	June 2020 – June 2021
·Created macroeconomic models to quantify business models of selected firms for FINRA Rule 2232.
             BNY Mellon, Pittsburgh, PA						                 September 2019 – June 2020
Corporate Trust Associate
·Leader & instructor of the Bloomberg software for the Corporate Trust Team.
LendingHome, Internship, Pittsburgh, PA
Funding Specialist & Post Closing Member	March 2019 – August 2019
·Reviewed closing documents to ensure precise execution for funding staff.
              Veraction/Trax, Junior Business Analyst, Memphis, TN	  June 2016 - August 2016                                                    June 2016 - August 2016
·Led the creation, development, and implementation of Request For Proposal (RFP) database.
"""


In [155]:
#resume_text

***

In [156]:
from transformers import pipeline

In [162]:
ner_pipeline = pipeline('ner', model = "dbmdz/bert-large-cased-finetuned-conll03-english",
                       aggregation_strategy = 'simple')

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [163]:
entities = ner_pipeline(resume_text)

## BERT

In [164]:
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Science, Label: MISC, Score: 0.7188
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##tics, Label: MISC, Score: 0.6904
Entity: MS, Label: MISC, Score: 0.6772
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: Science, Label: MISC, Score: 0.6226
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: Python, Label: MISC, Score: 0.7331
Entity: B, Label: MISC, Score: 0.3749
Entity: S, Label: MISC, Score: 0.4090
Entity: N, Label: MISC, Score: 0.3621
Entity: Networks, Label: MISC, Score:

In [165]:
filtered_entities = [entity for entity in entities if entity['entity_group'] != 'MISC']

In [166]:
for entity in filtered_entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: AWS, Label: ORG, Score: 0.8358
Entity: FINRA, Label: ORG, Score: 0.9555
Entity: Washington, Label: LOC, Score: 0.9981
Entity: D, Label: LOC, Score: 0.9980
Entity: C, Label: LOC, Score: 0.9992
Entity: Regulation, Label: ORG, Score: 0.4722
Entity: Power, Label: ORG, Score: 0.4565
Entity: Table, Label: ORG, Score: 0.5572
Entity: FINRA, Label: ORG, Score: 0.

In [167]:
import re

# Regex pattern for email extraction
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

# Find all email addresses in the text
emails = re.findall(email_pattern, resume_text)

# Display found email addresses
print(emails)

['Nicholas.E.Reese15@gmail.com']


In [168]:
# Load the NER pipeline
ner_pipeline2 = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")


# Detect entities using the NER pipeline
entities2 = ner_pipeline2(resume_text)

# Regex pattern for email extraction
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

# Find all email addresses in the text
emails = re.findall(email_pattern, resume_text)

# Print detected entities from NER
for entity in entities2:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")



Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Entity: Nicholas E, Label: PER, Score: 0.8347
Entity: Reese, Label: PER, Score: 0.8619
Entity: Van Ness, Label: LOC, Score: 0.6532
Entity: Washington D, Label: LOC, Score: 0.9981
Entity: C, Label: LOC, Score: 0.9990
Entity: Nicholas, Label: PER, Score: 0.9735
Entity: E, Label: PER, Score: 0.6905
Entity: Reese, Label: PER, Score: 0.9786
Entity: Georgetown University, Label: ORG, Score: 0.9974
Entity: Science, Label: MISC, Score: 0.7188
Entity: Business, Label: ORG, Score: 0.3938
Entity: ##tics, Label: MISC, Score: 0.6904
Entity: MS, Label: MISC, Score: 0.6772
Entity: ##BA, Label: ORG, Score: 0.6136
Entity: Dickinson College, Label: ORG, Score: 0.9974
Entity: & Political, Label: ORG, Score: 0.6406
Entity: Science, Label: MISC, Score: 0.6226
Entity: University of Bologna, Label: ORG, Score: 0.9910
Entity: Python, Label: MISC, Score: 0.7331
Entity: B, Label: MISC, Score: 0.3749
Entity: S, Label: MISC, Score: 0.4090
Entity: N, Label: MISC, Score: 0.3621
Entity: Networks, Label: MISC, Score:

test

In [169]:
resume_test = """
Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
Nicholas.E.Reese@gmail.com
"""

In [170]:
entities = ner_pipeline(resume_test)

In [171]:
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E. Reese Van Ness, Label: PER, Score: 0.8019
Entity: Washington D, Label: LOC, Score: 0.9986
Entity: C, Label: LOC, Score: 0.9995
Entity: Nicholas, Label: PER, Score: 0.9980
Entity: E. Reese, Label: PER, Score: 0.9686


In [172]:
thresholds ={
    'PER': .7,
    'ORG': .8
}

In [173]:
redacted_text = resume_test

In [174]:
for entity in entities:
    entity_type = entity['entity_group']
    score = entity['score']
    
    if entity_type == entity and score >= thresholds[entity]:
        entity_text = entity['word']
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)

# Redact the email addresses
redacted_text = re.sub(r'\S+@\S+', '[Redacted Email]', redacted_text)



print(redacted_text)


Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
[Redacted Email]



In [175]:
#print(redacted_text)

In [176]:
resume_test = """
Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
Nicholas.E.Reese@gmail.com
"""

In [177]:
entities = ner_pipeline(resume_test)

In [178]:
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Entity: Nicholas E. Reese Van Ness, Label: PER, Score: 0.8019
Entity: Washington D, Label: LOC, Score: 0.9986
Entity: C, Label: LOC, Score: 0.9995
Entity: Nicholas, Label: PER, Score: 0.9980
Entity: E. Reese, Label: PER, Score: 0.9686


In [179]:
thresholds ={
    'PER': .8,
    'ORG': .9,
    'LOC': .8
}

In [180]:
redacted_text = resume_test

In [181]:
entities_sorted = sorted(entities, key=lambda x: len(x['word']), reverse=True)


for entity in entities_sorted:
    entity_type = entity['entity_group']
    score = entity['score']
    
    if entity_type == entity_type and score >= thresholds[entity_type]:
        entity_text = entity['word']
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)


        
for entity in entities_sorted:
    entity_type = entity['entity_group']
    score = entity['score']
    
    if entity_type in thresholds and score >= thresholds[entity_type]:
        entity_text = entity['word']
        # Make sure to redact the smaller parts again if needed
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)

# Redact the email addresses
redacted_text = re.sub(r'\S+@\S+', '[Redacted Email]', redacted_text)
redacted_text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '[Redacted Phone]', redacted_text)

In [182]:
print(redacted_text)


[Redacted] [Redacted]
Van Ness, [Redacted].[Redacted].
[Redacted Phone]
[Redacted Email]



# Dezmond Initial Reactions

So far it looks like a single resume has been used for extracting entities and identifying emails.  The model that was used was NER Bert Model.  I think this was done to ensure each step worked before moving forward.

The process seems straight forward. But I'd like more resumes for exposure to different resume format, styles, entities and pii values.

For the sake of this exercise, I will do the following:
- add more regex patterns to help extract pii not captured in entities
- use other ner models to detect pii with entities
- MAYBE compare ner model results

I will make sure not to use a code editing tool to clean up my code or comments as to keep my points and intentions straightforward

## What Pii Values do resumes have that need to be identified
- phone number
- website link (linkedin, social media, git links)
- address
- name
- unique identifiers (tax id #, social security #, etc...)
- international identifiers should be considered (Mentioned in Deloitte Meeting)


## If Pii is a concern, how do we get rid or at least hide it
- redact
- replace


## What other EDA can I do?




### Other Notes,

I recoemmedn keeping a running list of all sites where you pull code from directly or reference.  I will add linked to sites i directly reference.

# More Dezmond Reactions

- So far I have performed test with 6 ner models
- It should be noted entity labels are different due to the NER models used ie per entity is broken down into i-per and b-per
- Spacy appears to be the cleanest when identifying and redacting entities (person, org, location) from a test
- All models appear succesffully redact entities but there are quite a few with multiple intersecting redact labels.  This needs to fixed so only redacted text replaced the entity or pii value
- All the thresholds were left at .75 for the tests.
- Dezmond's resume include linked, so I will test out the regex linked redacter


Next Steps:
- I will perform tests on the corpus (all saxa 4 resumes)
- I will add regex patterns for pii values not covered by ner enties
- I have to find a way to clean up the multiple redacted inputs when redacting entities


- If I had to pick a model today, I'd say spacy with Bert-large-NeR due to the cleanliness of the redacted outputs in tests.


# Regex patterns and ner model apis

## Other regex patterns for pii values in resumes

In [68]:
#us_full_name_pattern = r'\b[A-Z][a-z]*\s[A-Z][a-z]*\b',  # pattern for simpler full name (e.g., John Doe)
#international_full_name =

us_phone_number = r'(\+?1\s?)?(\(?\d{3}\)?[\s-]?)?\d{3}[\s-]?\d{4}',  # US phone # pattern
international_phone = r'(\+?\d{1,3}[-.\s]?)?(\(?\d{1,4}?\)?[-.\s]?)?(\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9})'
#us_address = r'\d{1,4}\s\w+\s\w+\.?',  # street address ('123 Main St) '' - Legacy
us_address = r'([0-9]+)\s([A-Za-z0-9\-\s]+),\s([A-Za-z\s]+),\s([A-Z]{2})\s([0-9\-]{5,10})'
international_address = r'^\d{1,5}\s[\w\s.,\'-]+,\s*[\w\s.,\'-]+,\s*[A-Z]{2,3}\s*\d{2,5}(-\d{4})?$'
social_security_number = r'\b\d{3}-\d{2}-\d{4}\b',  # Social security number pattern (XXX-XX-XXXX)
linkedin = r'\b[a-zA-Z0-9._-]+/in/[a-zA-Z0-9._-]+\b',  # LinkedIn profile



## Other ner models for detecting pii with entities

In [64]:
# xlm-roberta-large-NER
# https://huggingface.co/51la5/roberta-large-NER
# The model is a language model. The model can be used for token classification, a natural language understanding task in which a label is assigned to some tokens in a text.

from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
classifier = pipeline("ner", model=model, tokenizer=tokenizer)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [62]:
#albert-base-v2-finetuned-ner
#https://huggingface.co/Jorgeutd/albert-base-v2-finetuned-ner

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jorgeutd/albert-base-v2-finetuned-ner")
model = AutoModelForTokenClassification.from_pretrained("Jorgeutd/albert-base-v2-finetuned-ner")

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

In [59]:
#bert-base-NER
# https://huggingface.co/dslim/bert-base-NER
# bert-base-NER is a fine-tuned BERT model that is ready to use for Named Entity Recognition and achieves state-of-the-art performance for the NER task. It has been trained to recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

ner_pipeline2 = pipeline("ner", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [60]:
# distilbert-NER
# https://huggingface.co/dslim/distilbert-NER
# smaller, faster, more efficent variant of BERT
# distilbert-NER is the fine-tuned version of DistilBERT, which is a distilled variant of the BERT model. DistilBERT has fewer parameters than BERT, making it smaller, faster, and more efficient. distilbert-NER is specifically fine-tuned for the task of Named Entity Recognition (NER).

tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

In [61]:
# bert-large-NER
# https://huggingface.co/dslim/bert-large-NER
# bert-large-NER is a fine-tuned BERT model that is ready to use for Named Entity Recognition and achieves state-of-the-art performance for the NER task. It has been trained to recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [65]:
# spacy
 # https://ubiai.tools/fine-tuning-spacy-models-customizing-named-entity-recognition/
# As an open-source library, SpaCy provides pre-trained models for essential tasks like part-of-speech tagging, named entity recognition, and dependency parsing.
import spacy
#!python -m spacy download en_core_web_lg

nlp = spacy.load("en_core_web_lg") 
ner_pipeline = spacy.load("en_core_web_lg")

     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     -------------------------------------- 0.0/587.7 MB 326.8 kB/s eta 0:29:59
     -------------------------------------- 0.1/587.7 MB 798.9 kB/s eta 0:12:16
     ---------------------------------------- 0.3/587.7 MB 1.8 MB/s eta 0:05:19
     ---------------------------------------- 0.6/587.7 MB 2.7 MB/s eta 0:03:41
     ---------------------------------------- 0.9/587.7 MB 3.5 MB/s eta 0:02:49
     ---------------------------------------- 1.2/587.7 MB 4.1 MB/s eta 0:02:22
     ---------------------------------------- 1.5/587.7 MB 4.9 MB/s eta 0:02:01
     ---------------------------------------- 1.9/587.7 MB 5.3 MB/s eta 0:01:51
     ---------------------------------------- 2.4/587.7 MB 6.0 MB/s eta 0:01:37
     ---------------------------------------- 2.7/587.7 MB 6.

## ner model & regex pii identfying code

In [133]:
#looking at how well each ner model or regex pattern identified pii values
# Maybe figureout if we can set a threshold score for classifying entities .75 and up maybe...


import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline



# NER MODELS

#albert-base-v2-finetuned-ner
#tokenizer = AutoTokenizer.from_pretrained("Jorgeutd/albert-base-v2-finetuned-ner")
#model = AutoModelForTokenClassification.from_pretrained("Jorgeutd/albert-base-v2-finetuned-ner")

# xlm-roberta-large-NER
#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
#model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
#classifier = pipeline("ner", model=model, tokenizer=tokenizer)

# bert-base-NER
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

# distilbert-NER
# tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

# # bert-large-NER
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

# spacy
nlp = spacy.load("en_core_web_lg") 
ner_pipeline = spacy.load("en_core_web_lg")


# PIPELINE
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Finding entities using the NER pipeline
entities = ner_pipeline(resume_text)

# Regex pattern for email, full name, phone number, address, ssn, linkedin extraction
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
#us_phone_number_pattern = r'(\+?1\s?)?(\(?\d{3}\)?[\s-]?)?\d{3}[\s-]?\d{4}'  # US phone number pattern
#international_phone_pattern = r'(\+?\d{1,3}[-.\s]?)?(\(?\d{1,4}?\)?[-.\s]?)?(\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9})'  # International phone number pattern
#us_address_pattern = r'([0-9]+)\s([A-Za-z0-9\-\s]+),\s([A-Za-z\s]+),\s([A-Z]{2})\s([0-9\-]{5,10})'  # US address pattern
#international_address_pattern = r'^\d{1,5}\s[\w\s.,\'-]+,\s*[\w\s.,\'-]+,\s*[A-Z]{2,3}\s*\d{2,5}(-\d{4})?$'  # International address pattern
#social_security_number_pattern = r'\b\d{3}-\d{2}-\d{4}\b'  # Social Security Number pattern (XXX-XX-XXXX)
#linkedin_pattern = r'\b(?:https?://)?(?:www\.)?linkedin\.com/in/[a-zA-Z0-9._-]+\b'  # LinkedIn profile pattern


# Find all emails, full name, phone number, address, ssn, linkedin extraction in the text
emails = re.findall(email_pattern, resume_text)
#phones = re.findall(us_phone_number_pattern, resume_text)
#international_phones = re.findall(international_phone_pattern, resume_text)
#us_addresses = re.findall(us_address_pattern, resume_text)
#international_addresses = re.findall(international_address_pattern, resume_text, re.MULTILINE)
#ssns = re.findall(social_security_number_pattern, resume_text)
#linkedins = re.findall(linkedin_pattern, resume_text)

# Print detected entities from NER
entities = ner_pipeline(resume_text)
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']:.4f}")

# print entities extracted
print(f"Emails: {emails}")
#print(f"US Phone Numbers: {phones}")
#print(f"International Phone Numbers: {international_phones}")
#print(f"US Addresses: {us_addresses}")
#print(f"International Addresses: {international_addresses}")
#print(f"Social Security Numbers: {ssns}")
#print(f"LinkedIn Profiles: {linkedins}")

Entity: ▁nicholas, Label: B-PER, Score: 0.9945
Entity: ▁, Label: I-PER, Score: 0.9460
Entity: e, Label: I-PER, Score: 0.9031
Entity: ., Label: I-PER, Score: 0.9071
Entity: ▁reese, Label: I-PER, Score: 0.9656
Entity: ▁van, Label: I-PER, Score: 0.6648
Entity: ness, Label: I-ORG, Score: 0.4696
Entity: ▁washington, Label: B-LOC, Score: 0.9953
Entity: ▁, Label: I-LOC, Score: 0.9586
Entity: d, Label: I-LOC, Score: 0.8595
Entity: ., Label: I-LOC, Score: 0.8976
Entity: c, Label: I-LOC, Score: 0.8098
Entity: ., Label: I-LOC, Score: 0.7961
Entity: george, Label: B-ORG, Score: 0.9614
Entity: town, Label: B-ORG, Score: 0.9687
Entity: ▁university, Label: I-ORG, Score: 0.9749
Entity: ▁of, Label: I-MISC, Score: 0.4751
Entity: ▁science, Label: I-MISC, Score: 0.3806
Entity: ▁business, Label: I-MISC, Score: 0.5165
Entity: ▁ms, Label: B-ORG, Score: 0.9876
Entity: ba, Label: B-ORG, Score: 0.9873
Entity: dick, Label: B-ORG, Score: 0.9573
Entity: in, Label: B-ORG, Score: 0.9674
Entity: son, Label: B-ORG, Sc

# ner model entity redaction code & observations

In [145]:
from transformers import pipeline

# NER MODELS

#albert-base-v2-finetuned-ner
#tokenizer = AutoTokenizer.from_pretrained("Jorgeutd/albert-base-v2-finetuned-ner")
#model = AutoModelForTokenClassification.from_pretrained("Jorgeutd/albert-base-v2-finetuned-ner")

# xlm-roberta-large-NER
#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
#model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
#classifier = pipeline("ner", model=model, tokenizer=tokenizer)

# bert-base-NER
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

# distilbert-NER
tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

# # bert-large-NER
#tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
#model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

# spacy
#nlp = spacy.load("en_core_web_lg") 
#ner_pipeline = spacy.load("en_core_web_lg")


# PIPELINE
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)



def redact_pii(text):
    entities = ner_pipeline(text)
    redacted_text = text
    for entity in entities:
        #if entity['entity'] in ['B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']:
        if entity['entity'] in ['I-PER', 'I-LOC', 'I-ORG']:
        #if entity['entity'] in ['B-PER', 'I-PER']:
        #if entity['entity'] in ['B-LOC', 'I-LOC']:
        #if entity['entity'] in ['B-ORG', 'I-ORG']:
            start, end = entity['start'], entity['end']
            redacted_text = redacted_text[:start] + "[REDACTED]" + redacted_text[end:]
    
    return redacted_text

redacted_text_out = redact_pii(resume_text)

print(redacted_text_out)



Nicholas [[R[REDAC[RED[[REDACTED]ACTED]CTED]]ED]EDACTED]. Reese
Van [REDA[REDACTED]ED] Washington D.C.
412-216[REDACTED]olas.E.Reese15@gmail.com 
Education 
•Georgetown University - August 2023 – December 2024
Master of Science Business Analytics – MS[REDACTED]idate
Peer-Elected Class Representative 
•Dickinson College - August 2014 – September 2018
Economics & Polit[RE[REDACTED]al Science Double Major
Varsity Tennis Captain 
•University of Bol[REDACTED]ugust 2016

Skills
- R studio
- Python
- [REDAC[REDACTED]- SQL
- Neural Networks
- Machine Learning
- Macro Modeling
- AWS Cloud Services
- Pandas
- A/B Testing
- Scikit-Learn
- Econometrics

Professional Experience
FINRA, Washington, D.C.	June 2020 – Present 
Senior Analyst, Market Regulation	September 2022 – Present 
•Earned top 10% of performance of analysts for past two years.
•Led team in research implementing PostgreSQL and NoSQL queries for large data pulls.
•Spearheaded new analytical approaches to financial workflow with tool

In [138]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_lg") 
ner_pipeline = spacy.load("en_core_web_lg") # Spacy



def redact_pii(text):
    entities = ner_pipeline(text)
    redacted_text = text
    for entity in entities:
# Add and remove '#' to test different entity for redaction
        if entity['entity_group'] in ['PERSON', 'LOC','GPE', 'ORG']:
        #if entity['entity_group'] in ['MISC']:
        #if entity['entity_group'] in ['PERSON']:
        #if entity['entity_group'] in ['ORG']:
        #if entity['entity_group'] in ['LOC']:
        #if entity['entity_group'] in ['GPE']:
          redacted_text = redacted_text.replace(entity['word'], "[REDACTED]", 1)  # Replace only the first occurrence
            
def redact_pii(text):
    doc = nlp(text)
    redacted_text = text
    for ent in doc.ents:
        if ent.label_ in ['PERSON', 'LOC','GPE']:
        #if ent.label_ in ['MISC']:
       # if ent.label_ in ['PERSON']:
        #if ent.label_ in ['ORG']:
       # if ent.label_ in ['LOC']:
       # if ent.label_ in ['GPE']:
            redacted_text = redacted_text.replace(ent.text, "[REDACTED]", 1)  # Replace only the first occurrence
    
    return redacted_text

redacted_text_out = redact_pii(resume_text)

print(redacted_text_out)



[REDACTED]
[REDACTED], [REDACTED]
412-216-2398
Nicholas.E.Reese15@gmail.com 
Education 
•Georgetown University - August 2023 – December 2024
Master of Science Business Analytics – MSBA candidate
Peer-Elected Class Representative 
•Dickinson College - August 2014 – September 2018
Economics & Political Science Double Major
Varsity Tennis Captain 
•University of [REDACTED] - August 2016

Skills
- R studio
- Python
- Power BI
- SQL
- Neural Networks
- Machine Learning
- Macro Modeling
- AWS Cloud Services
- Pandas
- A/B Testing
- Scikit-Learn
- Econometrics

Professional Experience
FINRA, [REDACTED], [REDACTED]	June 2020 – Present 
Senior Analyst, Market Regulation	September 2022 – Present 
•Earned top 10% of performance of analysts for past two years.
•Led team in research implementing PostgreSQL and NoSQL queries for large data pulls.
•Spearheaded new analytical approaches to financial workflow with tools such as R & Python for model development.
•Developed working predictive modeling 

# Testing NER Models

In [242]:
from transformers import pipeline

# NER MODELS

#albert-base-v2-finetuned-ner
#tokenizer = AutoTokenizer.from_pretrained("Jorgeutd/albert-base-v2-finetuned-ner")
#model = AutoModelForTokenClassification.from_pretrained("Jorgeutd/albert-base-v2-finetuned-ner")

# xlm-roberta-large-NER
#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
#model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")
#classifier = pipeline("ner", model=model, tokenizer=tokenizer)

# bert-base-NER
#tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
#model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

# distilbert-NER
#tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
#model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

# # bert-large-NER
#tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
#model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

# spacy
nlp = spacy.load("en_core_web_lg") 
ner_pipeline = spacy.load("en_core_web_lg")


# PIPELINE
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

In [243]:
resume_test = """
Nicholas E. Reese
Van Ness, Washington D.C.
412-216-2398
Nicholas.E.Reese@gmail.com
"""



In [244]:
entities = ner_pipeline(resume_test)

In [245]:
for entity in entities:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']:.4f}")

Entity: Nicholas, Label: B-PER, Score: 0.9965
Entity: E, Label: I-PER, Score: 0.9526
Entity: ., Label: I-PER, Score: 0.6042
Entity: Reese, Label: I-PER, Score: 0.9913
Entity: Van, Label: B-LOC, Score: 0.9456
Entity: N, Label: I-LOC, Score: 0.9914
Entity: ##ess, Label: I-LOC, Score: 0.9914
Entity: Washington, Label: B-LOC, Score: 0.9980
Entity: D, Label: I-LOC, Score: 0.9984
Entity: ., Label: I-LOC, Score: 0.9859
Entity: C, Label: I-LOC, Score: 0.9987
Entity: Nicholas, Label: B-PER, Score: 0.9959
Entity: ., Label: I-PER, Score: 0.6760
Entity: E, Label: I-PER, Score: 0.9329
Entity: Reese, Label: I-PER, Score: 0.9931


In [246]:
thresholds = {
    'B-PER': 0.75,
    'I-PER': 0.75,
    'B-ORG': 0.75,
    'I-ORG': 0.75,
    'B-LOC': 0.75,
    'I-LOC': 0.75,
}

In [247]:
redacted_text = resume_test

In [248]:
entities_sorted = sorted(entities, key=lambda x: len(x['word']), reverse=True)


for entity in entities_sorted:
    entity_type = entity['entity']
    score = entity['score']
    
    if entity_type == entity_type and score >= thresholds[entity_type]:
        entity_text = entity['word']
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)


        
for entity in entities_sorted:
    entity_type = entity['entity']
    score = entity['score']
    
    if entity_type in thresholds and score >= thresholds[entity_type]:
        entity_text = entity['word']
        # Make sure to redact the smaller parts again if needed
        redacted_text = re.sub(re.escape(entity_text), '[Redacted]', redacted_text)

# Redact the email addresses
redacted_text = re.sub(r'\S+@\S+', '[Redacted Email]', redacted_text)
redacted_text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '[Redacted Phone]', redacted_text)

In [249]:
print(redacted_text)


[Redacted] [Redacted][Redacted] [Redacted]
[Redacted] [Redacted]ess, [Redacted] [Redacted][Redacted][Redacted][Redacted]
[Redacted Phone]
[Redacted Email]



## ALBERT Entity Redaction Observations(Entire Resume):



## ROBERTA Entity Redaction Observations(Entire Resume):


## Bert Entity Redaction Observations(Entire Resume):



## DISTILBERT Entity Redaction Observations(Entire Resume):



## BERT LARGE Entity Redaction Observations(Entire Resume):




## SpaCy Entity Redaction Observations(Entire Resume):



## Pattern Based Redaction Code & Observations (Entire Resume):

In [32]:
# redactomg pii values based on regex pattern
# Be consistent when running code for each pattern, add 

import re

#email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}' #email
us_phone_number_pattern = r'(\+?1\s?)?(\(?\d{3}\)?[\s-]?)?\d{3}[\s-]?\d{4}'  #US phone number pattern
#international_phone_pattern = r'(\+?\d{1,3}[-.\s]?)?(\(?\d{1,4}?\)?[-.\s]?)?(\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9})'  #International phone number pattern
#us_address_pattern = r'([0-9]+)\s([A-Za-z0-9\-\s]+),\s([A-Za-z\s]+),\s([A-Z]{2})\s([0-9\-]{5,10})'  #US address pattern
#international_address_pattern = r'^\d{1,5}\s[\w\s.,\'-]+,\s*[\w\s.,\'-]+,\s*[A-Z]{2,3}\s*\d{2,5}(-\d{4})?$'  #International address pattern
#social_security_number_pattern = r'\b\d{3}-\d{2}-\d{4}\b'  #Social Security Number pattern (XXX-XX-XXXX)
#linkedin_pattern = r'\b(?:https?://)?(?:www\.)?linkedin\.com/in/[a-zA-Z0-9._-]+\b'  #LinkedIn profile pattern

def redact_pattern_pii(text):
    #redacted_text = re.sub(email_pattern, "[REDACTED]", text)
    redacted_text = re.sub(us_phone_number_pattern, "[REDACTED]", text)
   # redacted_text = re.sub(international_phone_pattern, "[REDACTED]", text)
   # redacted_text = re.sub(us_address_pattern, "[REDACTED]", text)
   # redacted_text = re.sub(international_address_pattern, "[REDACTED]", text)
   # redacted_text = re.sub(social_security_number_pattern, "[REDACTED]", text)
   # redacted_text = re.sub(linkedin_pattern, "[REDACTED]", text)
    
    return redacted_text

redacted_text_out = redact_pattern_pii(resume_text)

print(redacted_text_out)



Nicholas E. Reese
Van Ness, Washington D.C.
[REDACTED]
Nicholas.E.Reese15@gmail.com 
Education 
•Georgetown University - August 2023 – December 2024
Master of Science Business Analytics – MSBA candidate
Peer-Elected Class Representative 
•Dickinson College - August 2014 – September 2018
Economics & Political Science Double Major
Varsity Tennis Captain 
•University of Bologna - August 2016

Skills
- R studio
- Python
- Power BI
- SQL
- Neural Networks
- Machine Learning
- Macro Modeling
- AWS Cloud Services
- Pandas
- A/B Testing
- Scikit-Learn
- Econometrics

Professional Experience
FINRA, Washington, D.C.	June 2020 – Present 
Senior Analyst, Market Regulation	September 2022 – Present 
•Earned top 10% of performance of analysts for past two years.
•Led team in research implementing PostgreSQL and NoSQL queries for large data pulls.
•Spearheaded new analytical approaches to financial workflow with tools such as R & Python for model development.
•Developed working predictive modeling s

## observations so far

Phone Numbers:
- incomplete phone numbers captured, tuples within parenthesis
- refine regex pattern

[\+(9[976]\d|8[987530]\d|6[987]\d|5[90]\d|42\d|3[875]\d|
2[98654321]\d|9[8543210]|8[6421]|6[6543210]|5[87654321]|
4[987654310]|3[9643210]|2[70]|7|1)\d{1,14}$] - found on https://stackoverflow.com/questions/2113908/what-regular-expression-will-match-valid-international-phone-numbers
- tried to 

Addresses:
- incomplete as well

https://stackoverflow.com/questions/55769704/regex-for-matching-a-single-line-standard-usps-address




# Miscellenious Notes

## Entity Types in SpaCy: 
https://towardsdatascience.com/extend-named-entity-recogniser-ner-to-label-new-entities-with-spacy-339ee5979044
* = relevent


**PERSON - People, including fictional.

ORP - Nationalities or religious or political groups.

FAC - Buildings, airports, highways, bridges, etc.

**ORG - Companies, agencies, institutions, etc.

**GPE - Countries, cities, states. *

**LOC - Non-GPE locations, mountain ranges, bodies of water. *

PRODUCT - Objects, vehicles, foods, etc. (Not services.)

EVENT - Named hurricanes, battles, wars, sports events, etc.

WORK_OF_ART - Titles of books, songs, etc.

LAW - Named documents made into laws.

LANGUAGE - Any named language.

DATE - Absolute or relative dates or periods.

TIME - Times smaller than a day.

PERCENT - Percentage, including "%".

MONEY - Monetary values, including unit.

QUANTITY - Measurements, as of weight or distance.

ORDINAL - "first", "second", etc.

CARDINAL - Numerals that do not fall under another type.

# Entity Output Bert-base NER, Distilbert-NER, Bert-large-NER, roberta-large-ner
Abbreviation	Description
O	Outside of a named entity

B-MISC	Beginning of a miscellaneous entity right after another miscellaneous entity

I-MISC	Miscellaneous entity

B-PER	Beginning of a person’s name right after another person’s name

I-PER	Person’s name

B-ORG	Beginning of an organization right after another organization

I-ORG	organization

B-LOC	Beginning of a location right after another location

I-LOC	Location